In [ ]:
from sklearn.metrics import classification_report
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=42)


In [ ]:
ori_data = pd.read_csv('./data/v2-4.csv', index_col=False)


In [ ]:
X = ori_data.drop(['Severity'], axis=1)
y = ori_data['Severity']


## Model

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from metrics import cross_valid, test


In [ ]:
x_train_valid, x_test, y_train_valid, y_test = train_test_split(X, y, test_size=0.2, stratify=y)


## Direct Testing

### AdaBoost


In [ ]:
# ada_clf = AdaBoostClassifier(n_estimators=100)
# ada_clf.fit(x_train_valid, y_train_valid)
# test(ada_clf, x_test, y_test)

## KFold cross validation


### AdaBoost

AdaBoost without balance dataset

In [ ]:
ada_clf = AdaBoostClassifier(n_estimators=100)
cross_valid(x_train_valid, y_train_valid, ada_clf, verbose=True, balance_cls=False)
test(ada_clf, x_test, y_test)

AdaBoost with balance dataset

In [ ]:
ada_clf = AdaBoostClassifier(n_estimators=100)
cross_valid(x_train_valid, y_train_valid, ada_clf, verbose=True, balance_cls=True)
test(ada_clf, x_test, y_test)
